<a href="https://colab.research.google.com/github/VamshiPoojari/MetaDataGenrate/blob/main/metadatagenerate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Firstly i would like to start by setting up an environment for reading text from PDFs and Word docs, Processing and analyzing that text with NLP techniques and Building and hosting a web interface (with Flask + ngrok) for user interaction

In [1]:
!pip install pdfplumber python-docx nltk spacy transformers sentence-transformers flask flask-ngrok pyngrok
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3

In [2]:
import pdfplumber
import docx
import nltk
import spacy
from transformers import pipeline, AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import torch
import re
import json
from datetime import datetime
import pandas as pd
from flask import Flask, request, render_template_string, jsonify
import os
from werkzeug.utils import secure_filename


This code block imports all the necessary libraries for a document processing and NLP web application. It includes tools to:


*   Extract text from PDFs (pdfplumber) and Word documents (docx)

*   Analyze and process language using NLP libraries like nltk, spacy, transformers, and sentence-transformers

*   Use deep learning models for tasks like summarization or embeddings (transformers, SentenceTransformer, torch)

*   Handle data structures and formatting (re, json, datetime, pandas)

*   Build a Flask web server to handle file uploads, form inputs, and return processed results via HTML or JSON (flask, werkzeug)

*   Manage file paths and storage securely (os, secure_filename)


In [3]:
 nltk.download('punkt', quiet=True)
 nltk.download('stopwords', quiet=True)
 nltk.download('averaged_perceptron_tagger', quiet=True)
 print("downloaded")

downloaded


This code downloads three essential language models from NLTK: punkt for breaking text into sentences and words, stopwords for filtering out common English words that don’t carry much meaning (like "the" or "is"), and averaged_perceptron_tagger for identifying parts of speech in text. The quiet=True argument hides extra output during downloads, and once all resources are ready, it prints “downloaded” to confirm completion.

In [4]:
nlp = spacy.load('en_core_web_sm')

This line loads English language model into object nlp using spaCy

In [5]:
#just a test
doc = nlp("This is a test sentence")
print(len(doc))
current_time = datetime.now()
print(current_time.strftime('%Y-%m-%d %H:%M:%S'))

5
2025-06-24 15:15:09


In [6]:
!pip install pytesseract pillow pdf2image opencv-python --quiet
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import cv2
import numpy as np


this block of code imports necessary libraries for performing OCR (Optical Character Recognition) on images and PDFs. Pillow (PIL) handles image manipulation, while pdf2image converts PDF pages into images so they can be processed by OCR.

In [7]:

class DocumentExtractor:


    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')

    def extract_from_pdf(self, pdf_path):
        text = ""
        try:
            with pdfplumber.open(pdf_path) as pdf:
                print(f" lenght: { len(pdf.pages) }")
                for page_num, page in enumerate(pdf.pages, 1):
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
                        print(f"  Page {page_num} processed")
                print(f" PDF extraction complete: {len(text)} characters extracted")
        except Exception as e:
            print(f" Error extracting PDF: {e}")
            return ""
        return text

    def extract_from_pdf_with_ocr(self, pdf_path):

        regular_text = self.extract_from_pdf(pdf_path)

        # Check if we got sufficient text
        word_count = len(regular_text.split()) if regular_text else 0

        if word_count > 30:
            print(f" PDF extraction successful: {word_count} words")
            return regular_text
        else:
            print(f" Limited text found ({word_count} words), switching to OCR...")
            return self._extract_with_ocr(pdf_path)

    def _extract_with_ocr(self, pdf_path):

        try:
            # limit to first 3 pages for performance
            pages = convert_from_path(pdf_path, dpi=200, first_page=1, last_page=3)

            ocr_text = ""
            for page_num, page_image in enumerate(pages, 1):
                print(f"  Processing page {page_num} with OCR...")

                # Extract text using OCR
                page_text = pytesseract.image_to_string(page_image, lang='eng', config='--psm 6')

                if page_text.strip():
                    ocr_text += f"{page_text}\n"
                    print(f"  Page {page_num}: {len(page_text.split())} words extracted")
                else:
                    print(f" Page {page_num}: No text found")

            if ocr_text.strip():
                total_words = len(ocr_text.split())
                print(f" OCR extraction complete: {total_words} words extracted")
                return ocr_text
            else:
                print(" no text")
                return ""

        except Exception as e:
            print(f" OCR extraction error: {e}")
            return self.extract_from_pdf(pdf_path)  # Fallback to regular method

    def extract_from_docx(self, docx_path):

        text = ""
        try:
            doc = docx.Document(docx_path)
            paragraph_count = len(doc.paragraphs)
            print(f"there are {paragraph_count} paragraphs")

            for i, paragraph in enumerate(doc.paragraphs, 1):
                if paragraph.text.strip():  # Only add non-empty paragraphs
                    text += paragraph.text + "\n"

            print(f"extraction complete: {len(text)} characters extracted")
        except Exception as e:
            print(f" Error extracting Word document: {e}")
            return ""
        return text

    def extract_from_txt(self, txt_path):
        try:
            with open(txt_path, 'r', encoding='utf-8') as file:
                text = file.read()
            print(f" Text file extraction complete: {len(text)} characters extracted")
            return text
        except Exception as e:
            print(f" Error extracting text file: {e}")
            return ""
    # extract_content with ocr and extract content have same code
    def extract_content_with_ocr(self, file_path):
        if not os.path.exists(file_path):
            print(f" File not found: {file_path}")
            return ""

        file_extension = file_path.lower().split('.')[-1]
        file_size = os.path.getsize(file_path)

        print(f" File: {os.path.basename(file_path)}")
        print(f"   File type: {file_extension.upper()}")
        print(f"   File size: {file_size:,} bytes ({file_size/1024:.1f} KB)")

        # Choose the right extraction method
        if file_extension == 'pdf':
            return self.extract_from_pdf_with_ocr(file_path)
        elif file_extension == 'docx':
            return self.extract_from_docx(file_path)
        elif file_extension == 'txt':
            return self.extract_from_txt(file_path)
        else:
            print(f" Unsupported file format: {file_extension}")
            print("   Supported formats: PDF, DOCX, TXT")
            return ""

    def extract_content(self, file_path):

        if not os.path.exists(file_path):
            print(f" File not found: {file_path}")
            return ""

        # Get file extension to determine file type
        file_extension = file_path.lower().split('.')[-1]
        file_size = os.path.getsize(file_path)

        print(f"File: {os.path.basename(file_path)}")
        print(f"   File type: {file_extension.upper()}")
        print(f"  File size: {file_size:,} bytes ({file_size/1024:.1f} KB)")

        # Choose the right extraction method with OCR support
        if file_extension == 'pdf':
            return self.extract_from_pdf_with_ocr(file_path)
        elif file_extension == 'docx':
            return self.extract_from_docx(file_path)
        elif file_extension == 'txt':
            return self.extract_from_txt(file_path)
        else:
            print(f" Unsupported file format: {file_extension}")
            print("   Supported formats: PDF, DOCX, TXT")
            return ""




extractor = DocumentExtractor()


test_content = """
Sample Document for Testing

This is a sample document created to test our automated metadata generation system with OCR support.

Key Information:
- Document Type: Test Document
- Created by: AI System
- Date: June 2025
- Purpose: Testing document extraction capabilities with OCR

The system should be able to extract this text and analyze it for metadata generation.
This includes identifying key concepts, generating summaries, and extracting relevant information.

OCR Integration: The system now supports both digital and scanned documents automatically.

Thank you for testing our enhanced system!
"""

test_file_path = 'test_content.txt'
with open(test_file_path, 'w', encoding='utf-8') as f:
    f.write(test_content)

print(f"\n sample: {test_file_path}")

extracted_text = extractor.extract_content(test_file_path)

if extracted_text:
    print(f"\n Test Results:")
    print(f"    Total characters: {len(extracted_text)}")
    print(f"    Total words: {len(extracted_text.split())}")
    print(f"    First 100 characters: {extracted_text[:100]}...")

else:
    print(" Extraction test failed")


os.remove(test_file_path)




 sample: test_content.txt
File: test_content.txt
   File type: TXT
  File size: 606 bytes (0.6 KB)
 Text file extraction complete: 606 characters extracted

 Test Results:
    Total characters: 606
    Total words: 89
    First 100 characters: 
Sample Document for Testing

This is a sample document created to test our automated metadata gener...


The script defines a DocumentExtractor class designed to extract text from various document formats—PDFs (with or without OCR), Word (.docx), and plain text (.txt) files. It uses libraries like pdfplumber for regular PDFs, pytesseract and pdf2image for OCR-based extraction from scanned PDFs, and python-docx for Word files. When a file is passed to extract_content or extract_content_with_ocr, the class identifies the file type and chooses the appropriate method to extract readable text. At the end of the script, a sample .txt file is created and processed using this extractor. The script prints out metadata such as word count and a preview of the extracted text. Finally, it cleans up by deleting the test file.

In [8]:
from collections import Counter


nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

class SemanticAnalyzer:


    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')

        from nltk.corpus import stopwords
        self.stop_words = set(stopwords.words('english'))

    def extract_key_entities(self, text):
        doc = self.nlp(text)
        entities = []

        for ent in doc.ents:
            if ent.label_ in ['PERSON', 'ORG', 'GPE', 'MONEY', 'DATE', 'EVENT']:
                entities.append({
                    'text': ent.text.strip(),
                    'label': ent.label_,
                    'description': spacy.explain(ent.label_)
                })

        unique_entities = []
        seen = set()
        for entity in entities:
            if entity['text'].lower() not in seen:
                unique_entities.append(entity)
                seen.add(entity['text'].lower())

        return unique_entities[:10]

    def generate_summary(self, text, max_sentences=3):
        if len(text) < 100:
            return text

        doc = self.nlp(text)
        sentences = [sent.text.strip() for sent in doc.sents if len(sent.text.strip()) > 20]

        if len(sentences) <= max_sentences:
            return ' '.join(sentences)

        word_frequencies = {}
        for word in doc:
            if not word.is_stop and not word.is_punct and len(word.text) > 2:
                word_frequencies[word.lemma_.lower()] = word_frequencies.get(word.lemma_.lower(), 0) + 1

        sentence_scores = {}
        for i, sentence in enumerate(sentences):
            sentence_doc = self.nlp(sentence)
            score = 0
            word_count = 0

            for word in sentence_doc:
                if not word.is_stop and not word.is_punct and len(word.text) > 2:
                    score += word_frequencies.get(word.lemma_.lower(), 0)
                    word_count += 1

            if word_count > 0:
                sentence_scores[i] = (score / word_count) + (len(sentences) - i) * 0.1

        top_sentences = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)[:max_sentences]
        top_sentences = sorted(top_sentences, key=lambda x: x[0])  # Sort by original order

        summary_sentences = [sentences[i] for i, score in top_sentences]
        return ' '.join(summary_sentences)

    def analyze_sentiment(self, text):
        positive_words = {'good', 'great', 'excellent', 'amazing', 'wonderful', 'fantastic',
                         'positive', 'success', 'successful', 'effective', 'efficient', 'improve',
                         'better', 'best', 'increase', 'growth', 'benefit', 'advantage'}

        negative_words = {'bad', 'terrible', 'awful', 'horrible', 'negative', 'fail', 'failure',
                         'poor', 'worse', 'worst', 'decrease', 'decline', 'problem', 'issue',
                         'difficult', 'challenge', 'risk', 'threat', 'disadvantage'}

        doc = self.nlp(text.lower())
        positive_count = 0
        negative_count = 0
        total_words = 0

        for token in doc:
            if not token.is_stop and not token.is_punct and len(token.text) > 2:
                total_words += 1
                if token.lemma_ in positive_words:
                    positive_count += 1
                elif token.lemma_ in negative_words:
                    negative_count += 1

        if positive_count > negative_count:
            sentiment = 'POSITIVE'
            confidence = min(0.9, 0.5 + (positive_count - negative_count) / total_words * 10)
        elif negative_count > positive_count:
            sentiment = 'NEGATIVE'
            confidence = min(0.9, 0.5 + (negative_count - positive_count) / total_words * 10)
        else:
            sentiment = 'NEUTRAL'
            confidence = 0.5

        return {
            'sentiment': sentiment,
            'confidence': round(confidence, 2)
        }

    def extract_keywords(self, text, top_k=10):
        """Extract the most important keywords from the document"""
        doc = self.nlp(text)

        keywords = []
        for token in doc:
            if (not token.is_stop and
                not token.is_punct and
                len(token.text) > 2 and
                token.pos_ in ['NOUN', 'ADJ', 'PROPN'] and
                token.lemma_.lower() not in self.stop_words):
                keywords.append(token.lemma_.lower())

        # Count frequency and return top keywords
        keyword_freq = Counter(keywords)
        return [word for word, freq in keyword_freq.most_common(top_k)]



analyzer = SemanticAnalyzer()

sample_text = """
Artificial Intelligence and Machine Learning Report

This comprehensive document provides an overview of artificial intelligence (AI) and machine learning (ML)
technologies in 2024. The report was prepared by Dr. Sarah Johnson at Stanford University.

Key findings include significant developments in the field:
- AI adoption has increased by 40% in enterprise environments, showing positive growth
- Machine learning models are becoming more efficient and effective
- Natural language processing shows promising results across various applications

The research was conducted between January and March 2024, involving 500 companies
across various industries including technology, healthcare, and finance. The study cost $2.5 million.

Major recommendations for organizations:
1. Invest in comprehensive AI training for employees
2. Implement gradual AI integration strategies
3. Focus on improving data quality and management

The findings demonstrate excellent progress in AI capabilities. However, some challenges remain
in terms of implementation costs and training requirements.

For more information about this groundbreaking research, contact the research team at Stanford University.
"""

print(f" sample document has {len(sample_text)} characters")

# Test all analyzer functions
entities = analyzer.extract_key_entities(sample_text)
summary = analyzer.generate_summary(sample_text)
sentiment = analyzer.analyze_sentiment(sample_text)
keywords = analyzer.extract_keywords(sample_text)


print(f"\n Extracted Entities {len(entities)} found:")
for entity in entities:
    print(f"   • {entity['text']} ({entity['label']}) - {entity['description']}")

print(f"\n Generated Summary:")
print(f"   {summary}")

print(f"\n Sentiment Analysis:")
print(f"   Sentiment: {sentiment['sentiment']}")
print(f"   Confidence: {sentiment['confidence']*100:.1f}%")

print(f"\n Keywords ({len(keywords)} found):")
print(f"   {', '.join(keywords)}")


 sample document has 1193 characters

 Extracted Entities 8 found:
   • Artificial Intelligence and Machine Learning Report (ORG) - Companies, agencies, institutions, etc.
   • ML (ORG) - Companies, agencies, institutions, etc.
   • 2024 (DATE) - Absolute or relative dates or periods
   • Sarah Johnson (PERSON) - People, including fictional
   • Stanford University (ORG) - Companies, agencies, institutions, etc.
   • between January and March 2024 (DATE) - Absolute or relative dates or periods
   • $2.5 million (MONEY) - Monetary values, including unit
   • AI (ORG) - Companies, agencies, institutions, etc.

 Generated Summary:
   Artificial Intelligence and Machine Learning Report

This comprehensive document provides an overview of artificial intelligence (AI) and machine learning (ML)
technologies in 2024. The report was prepared by Dr. Sarah Johnson at Stanford University. Key findings include significant developments in the field:
- AI adoption has increased by 40% in enterprise e

Semantic Analyser

This class analyzes the meaning and content of documents using spaCy and NLTK.
It can extract entities, keywords, create summaries, and analyze sentiment.

*    It includes a method to extract named entities such as people, organizations, locations, and dates, ensuring duplicates are removed.

*   Another method generates a summary by scoring sentences based on word
frequency and selecting the most relevant ones.
*   Sentiment analysis is done using predefined lists of positive and negative words, computing a basic confidence score to classify the text as positive, negative, or neutral.

*   The script also includes a keyword extraction method that identifies and returns the most frequent nouns, adjectives, and proper nouns after filtering out stopwords and punctuation.

To test these features, a sample text about artificial intelligence and machine learning is analyzed. The script prints out key entities, a summary, sentiment with confidence, and important keywords, demonstrating that the semantic analysis tools are functioning as expected.






In [9]:
class MetadataGenerator:

    def __init__(self):
        self.extractor = DocumentExtractor()
        self.analyzer = SemanticAnalyzer()

    def generate_metadata(self, file_path, filename):

        content = self.extractor.extract_content(file_path)

        if not content or len(content) < 10:
            return {
                "error": "Could not extract sufficient content from file",
                "filename": filename,
                "timestamp": datetime.now().isoformat()
            }

        print(f"   Content extracted: {len(content)} characters")

        try:
            file_stats = os.stat(file_path)
            file_info = {
                "filename": filename,
                "file_size_bytes": file_stats.st_size,
                "file_size_mb": round(file_stats.st_size / (1024*1024), 3),
                "file_size_kb": round(file_stats.st_size / 1024, 1),
                "created_date": datetime.fromtimestamp(file_stats.st_ctime).isoformat(),
                "modified_date": datetime.fromtimestamp(file_stats.st_mtime).isoformat(),
                "file_type": filename.split('.')[-1].upper() if '.' in filename else 'UNKNOWN'
            }
            print(f"   File info collected: {file_info['file_size_kb']} KB, {file_info['file_type']} format")
        except Exception as e:
            print(f"   Could not get file stats: {e}")
            file_info = {"filename": filename, "error": "Could not access file stats"}

        words = content.split()
        sentences = content.split('.')
        paragraphs = [p.strip() for p in content.split('\n\n') if p.strip()]

        content_stats = {
            "word_count": len(words),
            "character_count": len(content),
            "character_count_no_spaces": len(content.replace(' ', '')),
            "sentence_count": len([s for s in sentences if s.strip()]),
            "paragraph_count": len(paragraphs),
            "average_words_per_sentence": round(len(words) / max(1, len([s for s in sentences if s.strip()])), 1),
            "estimated_reading_time_minutes": round(len(words) / 200, 1),  # 200 words per minute average
            "readability_score": self._calculate_readability(content)
        }
        print(f"   Content stats: {content_stats['word_count']} words, {content_stats['sentence_count']} sentences")

        analysis_content = content[:3000] if len(content) > 3000 else content

        entities = self.analyzer.extract_key_entities(analysis_content)
        summary = self.analyzer.generate_summary(analysis_content)
        sentiment = self.analyzer.analyze_sentiment(analysis_content)
        keywords = self.analyzer.extract_keywords(analysis_content)

        print(f"Semantic analysis complete: {len(entities)} entities, {len(keywords)} keywords")

        # Calculate confidence score
        confidence_score = self._calculate_confidence_score(content, entities, keywords, content_stats)

        metadata = {
            "document_metadata": {
                "generation_info": {
                    "generated_at": datetime.now().isoformat(),
                    "generator_version": "1.0.0",
                    "processing_time_seconds": "< 1",
                    "confidence_score": confidence_score,
                    "analysis_completeness": "Full" if len(content) <= 3000 else "Partial (first 3000 chars)"
                }
            },

            "file_information": file_info,

            "content_analysis": content_stats,

            "semantic_analysis": {
                "document_summary": summary,
                "primary_keywords": keywords,
                "sentiment_analysis": sentiment,
                "named_entities": entities,
                "content_type": self._classify_content_type(content, entities, keywords),
                "main_topics": self._extract_main_topics(keywords, entities)
            },

            "quality_indicators": {
                "content_richness": "High" if len(words) > 500 else "Medium" if len(words) > 100 else "Low",
                "entity_density": round(len(entities) / max(1, len(words)) * 100, 2),
                "keyword_diversity": len(set(keywords)),
                "structural_complexity": "High" if content_stats['paragraph_count'] > 10 else "Medium" if content_stats['paragraph_count'] > 3 else "Low"
            }
        }

        print(f"   Confidence Score: {confidence_score*100:.1f}%")
        print(f"   Content Type: {metadata['semantic_analysis']['content_type']}")

        return metadata

    def _calculate_confidence_score(self, content, entities, keywords, content_stats):
        score = 0.3

        # Content length factor
        word_count = content_stats['word_count']
        if word_count > 500:
            score += 0.3
        elif word_count > 200:
            score += 0.2
        elif word_count > 50:
            score += 0.1

        # Entity richness factor
        if len(entities) > 5:
            score += 0.2
        elif len(entities) > 2:
            score += 0.15
        elif len(entities) > 0:
            score += 0.1

        # Keyword diversity factor
        if len(keywords) > 8:
            score += 0.15
        elif len(keywords) > 4:
            score += 0.1
        elif len(keywords) > 0:
            score += 0.05

        # Structure factor
        if content_stats['paragraph_count'] > 3:
            score += 0.1

        return min(1.0, round(score, 2))

    def _calculate_readability(self, text):

        words = len(text.split())
        sentences = len([s for s in text.split('.') if s.strip()])
        if sentences == 0:
            return "Unknown"

        avg_words_per_sentence = words / sentences
        if avg_words_per_sentence < 15:
            return "Easy"
        elif avg_words_per_sentence < 25:
            return "Medium"
        else:
            return "Complex"

    def _classify_content_type(self, content, entities, keywords):

        content_lower = content.lower()

        # Check for what sort of content it is whether it is a academic content ,business, technical or legal content
        academic_terms = ['research', 'study', 'analysis', 'university', 'findings', 'methodology']
        if any(term in content_lower for term in academic_terms):
            return "Academic/Research"

        business_terms = ['company', 'business', 'market', 'revenue', 'strategy', 'customers']
        if any(term in content_lower for term in business_terms):
            return "Business"

        tech_terms = ['system', 'technology', 'software', 'data', 'algorithm', 'technical']
        if any(term in content_lower for term in tech_terms):
            return "Technical"

        legal_terms = ['law', 'legal', 'contract', 'agreement', 'terms', 'conditions']
        if any(term in content_lower for term in legal_terms):
            return "Legal"

        return "General"

    def _extract_main_topics(self, keywords, entities):

        all_terms = keywords + [entity['text'].lower() for entity in entities]


        topics = []
        if any('ai' in term or 'artificial' in term or 'machine' in term for term in all_terms):
            topics.append("Artificial Intelligence")
        if any('business' in term or 'company' in term or 'market' in term for term in all_terms):
            topics.append("Business")
        if any('research' in term or 'study' in term or 'analysis' in term for term in all_terms):
            topics.append("Research")
        if any('technology' in term or 'technical' in term or 'system' in term for term in all_terms):
            topics.append("Technology")

        return topics if topics else ["General"]

#created metadata generator
generator = MetadataGenerator()

test_document = """
Artificial Intelligence in Healthcare: A Comprehensive Study

Executive Summary

This research report examines the implementation of artificial intelligence (AI) technologies
in healthcare systems across North America. The study was conducted by Dr. Emily Chen at
Massachusetts General Hospital in collaboration with Stanford University.

Background and Methodology

The research team analyzed data from 150 hospitals between January 2024 and May 2024.
The study involved a budget of $1.2 million and included interviews with 500 healthcare
professionals across various specialties.

Key Findings

Our analysis reveals significant improvements in patient outcomes through AI integration:

1. Diagnostic accuracy increased by 23% when AI systems were used alongside traditional methods
2. Patient wait times decreased by an average of 35 minutes
3. Treatment costs were reduced by 18% on average
4. Staff satisfaction improved due to reduced administrative burden

The most successful implementations were found in radiology, pathology, and emergency medicine.
Dr. Sarah Johnson, Chief of Radiology at Boston Medical Center, noted: "AI has revolutionized
our diagnostic capabilities while allowing us to focus more on patient care."

Challenges and Limitations

Despite positive outcomes, several challenges were identified:
- Initial implementation costs averaging $250,000 per department
- Training requirements for medical staff
- Data privacy and security concerns
- Regulatory compliance issues

Future Recommendations

Based on our findings, we recommend:
1. Gradual implementation starting with high-impact areas
2. Comprehensive staff training programs
3. Investment in robust cybersecurity measures
4. Collaboration with technology vendors for customized solutions

Conclusion

The integration of AI in healthcare shows tremendous promise for improving patient outcomes
and operational efficiency. However, successful implementation requires careful planning,
adequate resources, and strong leadership commitment.

This report was prepared by the Healthcare Innovation Research Team at Massachusetts General Hospital.
For additional information, contact Dr. Emily Chen at echen@mgh.harvard.edu.
"""

test_file_path = '/tmp/comprehensive_test.txt'
with open(test_file_path, 'w', encoding='utf-8') as f:
    f.write(test_document)

print(f" Created comprehensive test document ({len(test_document)} characters)")

metadata = generator.generate_metadata(test_file_path, 'comprehensive_test.txt')

print(f"\n RESULTS")

if 'error' not in metadata:
    print(f" FILE: {metadata['file_information']['filename']}")
    print(f" SIZE: {metadata['file_information']['file_size_kb']} KB")
    print(f" WORDS: {metadata['content_analysis']['word_count']}")
    print(f" READING TIME: {metadata['content_analysis']['estimated_reading_time_minutes']} minutes")
    print(f" CONTENT TYPE: {metadata['semantic_analysis']['content_type']}")
    print(f" SENTIMENT: {metadata['semantic_analysis']['sentiment_analysis']['sentiment']}")
    print(f" CONFIDENCE: {metadata['document_metadata']['generation_info']['confidence_score']*100:.1f}%")

    print(f"\n SUMMARY:")
    print(f"   {metadata['semantic_analysis']['document_summary'][:200]}...")

    print(f"\n TOP KEYWORDS:")
    print(f"   {', '.join(metadata['semantic_analysis']['primary_keywords'][:8])}")

    print(f"\n ENTITIES FOUND:")
    for entity in metadata['semantic_analysis']['named_entities'][:5]:
        print(f"   • {entity['text']} ({entity['label']})")

else:
    print(f" Error: {metadata['error']}")

os.remove(test_file_path)


 Created comprehensive test document (2204 characters)
File: comprehensive_test.txt
   File type: TXT
  File size: 2,204 bytes (2.2 KB)
 Text file extraction complete: 2204 characters extracted
   Content extracted: 2204 characters
   File info collected: 2.2 KB, TXT format
   Content stats: 292 words, 24 sentences
Semantic analysis complete: 10 entities, 10 keywords
   Confidence Score: 95.0%
   Content Type: Academic/Research

 RESULTS
 FILE: comprehensive_test.txt
 SIZE: 2.2 KB
 WORDS: 292
 READING TIME: 1.5 minutes
 CONTENT TYPE: Academic/Research
 SENTIMENT: POSITIVE
 CONFIDENCE: 95.0%

 SUMMARY:
   Artificial Intelligence in Healthcare: A Comprehensive Study

Executive Summary

This research report examines the implementation of artificial intelligence (AI) technologies
in healthcare systems acr...

 TOP KEYWORDS:
   healthcare, implementation, patient, study, research, dr., hospital, outcome

 ENTITIES FOUND:
   • Artificial Intelligence (ORG)
   • Healthcare (ORG)
   • Emily Ch

   This engine combines document extraction and semantic analysis to generate comprehensive metadata for any document.
   It collects basic file info (like size and creation date), content stats (word count, sentence count, readability), and performs semantic analysis to generate a summary, detect sentiment, extract keywords, and identify named entities like people and organizations. It also classifies the content type (e.g., academic, technical) and estimates a confidence score based on the document’s richness and structure.

In [10]:

class EnhancedMetadataGenerator(MetadataGenerator):

    def __init__(self):
        self.extractor = DocumentExtractor()
        self.analyzer = SemanticAnalyzer()

        self.extractor.extract_content = self.extractor.extract_content_with_ocr

    def generate_metadata(self, file_path, filename):
        print(f"\n Generating metadata with OCR support {filename}")

        metadata = super().generate_metadata(file_path, filename)

        if 'file_information' in metadata:
            metadata['file_information']['ocr_capable'] = "Yes"
            metadata['file_information']['extraction_method'] = "Regular + OCR Fallback"

        return metadata

metadata_generator = EnhancedMetadataGenerator()

print("all done")

all done


In [11]:
from flask import Flask, request, render_template_string, jsonify
from werkzeug.utils import secure_filename
import os
from datetime import datetime
import threading
import time

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = '/tmp/uploads'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB max file size
app.secret_key = 'metadata_generator_secret_key'

os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)


HTML_TEMPLATE_OCR = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title> AI Metadata Generator with OCR</title>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); min-height: 100vh; padding: 20px; }
        .container { max-width: 1000px; margin: 0 auto; background: white; border-radius: 20px; box-shadow: 0 20px 40px rgba(0,0,0,0.1); }
        .header { background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 40px; text-align: center; border-radius: 20px 20px 0 0; }
        .header h1 { font-size: 2.5em; margin-bottom: 10px; font-weight: 700; }
        .header p { font-size: 1.2em; opacity: 0.9; }
        .ocr-badge { background: rgba(255,255,255,0.2); padding: 8px 16px; border-radius: 20px; display: inline-block; margin-top: 10px; font-size: 0.9em; }
        .content { padding: 40px; }
        .upload-section { background: #f8f9fa; border: 3px dashed #dee2e6; border-radius: 15px; padding: 40px; text-align: center; margin-bottom: 30px; transition: all 0.3s ease; }
        .upload-section:hover { border-color: #667eea; background: #f0f4ff; }
        .file-input-button { padding: 15px 30px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 10px; cursor: pointer; border: none; font-size: 1.1em; font-weight: 600; }
        .file-input-button:hover {
            transform: translateY(-2px);
            box-shadow: 0 10px 20px rgba(102, 126, 234, 0.3);
        }
        .submit-btn { background: linear-gradient(135deg, #28a745 0%, #20c997 100%); color: white; padding: 15px 40px; border: none; border-radius: 10px; font-size: 1.1em; cursor: pointer; margin-top: 20px; }
        .results { background: #f8f9fa; padding: 30px; border-radius: 15px; margin-top: 30px; border-left: 5px solid #667eea; }
        .stat-grid { display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px; margin-top: 15px; }
        .stat-item { background: white; padding: 15px; border-radius: 8px; text-align: center; box-shadow: 0 2px 10px rgba(0,0,0,0.05); }
        .stat-value { font-size: 1.5em; font-weight: bold; color: #667eea; }
        .stat-label { color: #6c757d; font-size: 0.9em; margin-top: 5px; }
        .tag { display: inline-block; background: #e3f2fd; color: #1565c0; padding: 5px 12px; margin: 3px; border-radius: 20px; font-size: 0.9em; }
        .summary-box { background: #fff3cd; border: 1px solid #ffeaa7; border-radius: 10px; padding: 20px; margin: 15px 0; font-style: italic; }
        .error-message { background: #f8d7da; color: #721c24; padding: 20px; border-radius: 10px; margin-top: 20px; }
        .ocr-indicator { background: #d4edda; color: #155724; padding: 10px; border-radius: 5px; margin: 10px 0; }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1> AI Metadata Generator</h1>
            <p>Advanced document analysis with OCR support</p>
            <div class="ocr-badge"> OCR Enabled for Scanned Documents</div>
        </div>

        <div class="content">
            <form method="POST" enctype="multipart/form-data">
                <div class="upload-section">
                    <input type="file" name="file" accept=".pdf,.docx,.txt" required style="margin-bottom: 15px;">
                    <p> Upload PDF, DOCX, or TXT files</p>
                    <p style="font-size: 0.9em; color: #6c757d; margin-top: 10px;">
                         Maax Limit of size: 16 MB
                    </p>
                </div>
                <button type="submit" class="submit-btn"> Generate Metadata with AI</button>
            </form>

            {% if metadata %}
            <div class="results">
                <h2> AI-Generated Metadata</h2>

                {% if metadata.file_information.get('ocr_capable') == 'Yes' %}
                <div class="ocr-indicator">
                     <strong>OCR Capability:</strong> This system can process scanned documents automatically
                </div>
                {% endif %}

                <div class="stat-grid">
                    <div class="stat-item">
                        <div class="stat-value">{{ metadata.file_information.filename }}</div>
                        <div class="stat-label">Filename</div>
                    </div>
                    <div class="stat-item">
                        <div class="stat-value">{{ metadata.content_analysis.word_count }}</div>
                        <div class="stat-label">Words Extracted</div>
                    </div>
                    <div class="stat-item">
                        <div class="stat-value">{{ metadata.semantic_analysis.content_type }}</div>
                        <div class="stat-label">Content Type</div>
                    </div>
                    <div class="stat-item">
                        <div class="stat-value">{{ (metadata.document_metadata.generation_info.confidence_score * 100)|round(1) }}%</div>
                        <div class="stat-label">AI Confidence</div>
                    </div>
                </div>

                <div style="margin-top: 25px;">
                    <h3> AI Analysis Results</h3>
                    <div class="summary-box">
                        <strong>Summary:</strong> {{ metadata.semantic_analysis.document_summary }}
                    </div>

                    <p><strong> Keywords:</strong></p>
                    {% for keyword in metadata.semantic_analysis.primary_keywords %}
                        <span class="tag">{{ keyword }}</span>
                    {% endfor %}

                    <p style="margin-top: 15px;"><strong> Sentiment:</strong>
                        <span class="tag">{{ metadata.semantic_analysis.sentiment_analysis.sentiment }}</span>
                    </p>
                </div>
            </div>
            {% endif %}

            {% if error %}
            <div class="error-message">
                <strong> Error:</strong> {{ error }}
            </div>
            {% endif %}
        </div>
    </div>
</body>
</html>
'''

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return render_template_string(HTML_TEMPLATE_OCR, error='No file selected')

        file = request.files['file']
        if file.filename == '':
            return render_template_string(HTML_TEMPLATE_OCR, error='No file selected')

        if file:
            filename = secure_filename(file.filename)
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)

            try:
                file.save(file_path)
                print(f" Processing {filename} with OCR-enhanced system...")

                metadata = metadata_generator.generate_metadata(file_path, filename)

                os.remove(file_path)

                if 'error' in metadata:
                    return render_template_string(HTML_TEMPLATE_OCR, error=metadata['error'])

                print(f" Successfully processed {filename} ")
                return render_template_string(HTML_TEMPLATE_OCR, metadata=metadata)

            except Exception as e:
                if os.path.exists(file_path):
                    os.remove(file_path)
                return render_template_string(HTML_TEMPLATE_OCR, error=f'Error: {str(e)}')

    return render_template_string(HTML_TEMPLATE_OCR)


This Flask-based web application allows users to upload documents (PDF, DOCX, or TXT) and receive detailed AI-generated metadata. It supports both digital and scanned documents using OCR, making it useful for extracting information from a wide range of files. The system performs semantic analysis to generate summaries, keywords, sentiment, and more — all through a clean, user-friendly interface.

*   **Web App Setup**: Uses Flask to handle uploads of files up to 16MB, with a secure and configurable server environment.
*   **File Upload & Validation**: Accepts and validates user-uploaded files, storing them temporarily for processing.
*    **AI-Powered Metadata Extraction:** Employs an OCR-enhanced metadata generator to extract and analyze text, even from scanned PDFs or images.
*   **Smart Analysis:** Produces insights like word/sentence/paragraph counts, content type classification, document summaries, sentiment analysis, and key terms.
*  **File Cleanup:** Automatically deletes uploaded files after processing to keep the server clean and efficient.
*  **User Interface**: Presents results in a visually appealing web interface with responsive design, keyword tags, and summary boxes.






you have to sign in first to get your own ngrok Authenticatication code

In [12]:
from pyngrok import ngrok, conf
import threading
import time

ngrok.set_auth_token("2ypISdRIC82MDPJamtVYjnZFBzP_3vzZnAyRJwMguJzJ1WB6h")  #i have put my own

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()


time.sleep(3)

public_url = ngrok.connect(5000)
print(f"\n Public URL: {public_url}")
print(f" Share this link with anyone!")



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit



 Public URL: NgrokTunnel: "https://c16e-34-168-13-68.ngrok-free.app" -> "http://localhost:5000"
 Share this link with anyone!




* ngrok.set_auth_token(...) sets your personal Ngrok token to authenticate and allow tunnel creation via your Ngrok account.
*  **Runs Flask App in Background:** Defines a run_flask() function to start the Flask server locally on port 5000, then starts it in a background thread using Python's threading.
*  ngrok.connect(5000) creates a secure public URL that maps to your local Flask app on port 5000. And also displays the public Ngrok link in the console, which you can share to give others access to your local app over the internet.


In [13]:
from pyngrok import ngrok
import time

# Kill all existing ngrok tunnels
try:
    ngrok.kill()
    print(" All ngrok tunnels stopped")
except Exception as e:
    print(f" Tunnel cleanup: {e}")

# Wait a moment for cleanup
time.sleep(2)

# Set your authtoken again
ngrok.set_auth_token("2ypISdRIC82MDPJamtVYjnZFBzP_3vzZnAyRJwMguJzJ1WB6h")

# Start fresh Flask server on a different port
import threading
import time
from flask import Flask

# Create new app instance on different port
def run_flask_clean():
    try:
        print(" Starting clean Flask server on port 5001...")
        app.run(host='0.0.0.0', port=5001, debug=False, use_reloader=False)
    except Exception as e:
        print(f"Flask error: {e}")

# Start Flask on port 5001 instead of 5000 and wait for flask to start sleep 3 used hence
flask_thread = threading.Thread(target=run_flask_clean, daemon=True)
flask_thread.start()
time.sleep(3)

# Create clean tunnel
try:
    public_url = ngrok.connect(5001)  # Connect to port 5001
    print(f" NEW PUBLIC URL: {public_url}")
    print(f" Your OCR-enhanced metadata generator is live!")

    print(f"\n Open this URL to test your system: {public_url}")

except Exception as e:
    print(f" Error: {e}")
    print("\n Alternative solution:")
    print("1. Go to: https://dashboard.ngrok.com/status/tunnels")
    print("2. Manually stop all running tunnels")
    print("3. Try again with fresh setup")

print(f"\n Cleanup and restart complete!")


 All ngrok tunnels stopped
 Starting clean Flask server on port 5001...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.28.0.12:5001
INFO:werkzeug:Press CTRL+C to quit


 NEW PUBLIC URL: NgrokTunnel: "https://10d5-34-168-13-68.ngrok-free.app" -> "http://localhost:5001"
 Your OCR-enhanced metadata generator is live!

 Open this URL to test your system: NgrokTunnel: "https://10d5-34-168-13-68.ngrok-free.app" -> "http://localhost:5001"

 Cleanup and restart complete!
